In [ ]:
import networkx as nx
from collections import Counter
import numpy as np

G = nx.read_edgelist('/content/drive/MyDrive/SMA/web-Stanford.txt.gz', create_using = nx.DiGraph)

In [ ]:
print(G)

DiGraph with 281903 nodes and 2312497 edges


In [ ]:
G.is_directed()

True

# **Degree Based Sampling**

In [ ]:
import networkx as nx

# Initializing degrees dictionary to store total degree (in-degree + out-degree) of each node
degrees = {node: deg for node, deg in G.degree()}

# Sorting nodes by their total degree in descending order and selecting the top N nodes
N = 30500
top_nodes = sorted(degrees, key=degrees.get, reverse=True)[:N]

# Create a subgraph from the top nodes
subgraph = G.subgraph(top_nodes)

# If the subgraph is not strongly connected or has many strongly connected components, refining it
if not nx.is_strongly_connected(subgraph) or len(list(nx.strongly_connected_components(subgraph))) > 15:
    # Extracting the largest strongly connected component
    largest_component = max(nx.strongly_connected_components(subgraph), key=len)
    subgraph = G.subgraph(largest_component)

# Output basic information about the subgraph
print(f"Subgraph size: {subgraph.number_of_nodes()}")
print(f"Subgraph density: {nx.density(subgraph)}")
print(subgraph)

Subgraph size: 13142
Subgraph density: 0.002079421156189292
DiGraph with 13142 nodes and 359114 edges


In [ ]:
print(subgraph)

DiGraph with 13142 nodes and 359114 edges


**Subgraph**

In [ ]:
# Export the subgraph to a CSV file
nx.write_edgelist(subgraph, "/content/drive/MyDrive/SMA/subgraph.csv", delimiter=",", data=False)

# Add header row to the CSV file
with open("/content/drive/MyDrive/SMA/subgraph.csv", "r+") as f:
    content = f.read()
    f.seek(0, 0)
    f.write("source,target\n" + content)

In [ ]:
print(subgraph)

DiGraph with 13142 nodes and 359114 edges


# **Centrality Analysis**

In [ ]:
degree_centrality= nx.degree_centrality(subgraph)

print(sorted(degree_centrality.items(), key=lambda kv:(kv[1], kv[0]), reverse=True)[:10])

[('181701', 0.3581919184232555), ('247241', 0.3562133779773229), ('96745', 0.35613728026786395), ('77999', 0.35613728026786395), ('259455', 0.35613728026786395), ('221087', 0.35613728026786395), ('183004', 0.35613728026786395), ('17781', 0.35613728026786395), ('176790', 0.35613728026786395), ('137632', 0.35613728026786395)]


# **Page Rank Analysis**

In [ ]:
import numpy as np
import networkx as nx
from sklearn.metrics import mean_absolute_error, mean_squared_error
import scipy.sparse as sp

def adjacency_to_sparse_matrix(subgraph):
    n = len(subgraph.nodes())
    rows = []
    cols = []
    for edge in subgraph.edges():
        rows.append(edge[0])
        cols.append(edge[1])
    return sp.coo_matrix(([1]*len(rows),(rows,cols)), shape=(n,n))

def classic_pagerank(subgraph, alpha=0.85, max_iter=1000):
    n = subgraph.number_of_nodes()
    A = nx.adjacency_matrix(subgraph)
    D = sp.diags(np.array(A.sum(axis=1)).flatten())
    P = sp.linalg.inv(D).dot(A)
    x = np.ones((n, 1)) / n
    for i in range(max_iter):
        xlast = x
        x = alpha * P.dot(x) + (1 - alpha) / n
        if np.allclose(x, xlast):
            break
    return x, i+1

def heat_kernel_pagerank(subgraph, t=1, max_iter=1000):
    subgraph = nx.convert_node_labels_to_integers(subgraph)
    n = subgraph.number_of_nodes()
    A = adjacency_to_sparse_matrix(subgraph)
    D = sp.diags(np.array(A.sum(axis=1)).flatten())
    P = sp.linalg.inv(D).dot(A)
    Q = sp.csr_matrix(np.exp(-t * P.toarray()))
    K = np.array(Q.sum(axis=1)).flatten()
    H = sp.diags(1/K).dot(Q)
    pr = np.ones(n) / n
    for i in range(max_iter):
        old_pr = pr.copy()
        pr = H.dot(pr)
        if np.allclose(pr, old_pr, atol=1e-6):
            return pr, i+1
    return pr, max_iter

# Compute ground-truth PageRank scores
gt_pr = nx.pagerank(subgraph, alpha=0.85, tol=1e-6)

# Compute Classic PageRank scores
pr1, num_iter1 = classic_pagerank(subgraph, alpha=0.85, max_iter=1000)

# Compute Heat Kernel PageRank scores
pr2, num_iter2 = heat_kernel_pagerank(subgraph, t=1, max_iter=1000)

# Compute mean absolute error and root mean squared error for Classic PageRank
classic_mae = mean_absolute_error(list(gt_pr.values()), pr1)
classic_rmse = np.sqrt(mean_squared_error(list(gt_pr.values()), pr1))

# Compute mean absolute error and root mean squared error for Heat Kernel PageRank
heat_kernel_mae = mean_absolute_error(list(gt_pr.values()), pr2)
heat_kernel_rmse = np.sqrt(mean_squared_error(list(gt_pr.values()), pr2))


/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_dsolve/linsolve.py:229: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


In [ ]:
print(f"Classic PageRank MAE: {classic_mae:.6f}")
print(f"Classic PageRank RMSE: {classic_rmse:.6f}")

print(f"Heat Kernel PageRank MAE: {heat_kernel_mae:.6f}")
print(f"Heat Kernel PageRank RMSE: {heat_kernel_rmse:.6f}")

Classic PageRank MAE: 0.000093
Classic PageRank RMSE: 0.000492
Heat Kernel PageRank MAE: 0.000093
Heat Kernel PageRank RMSE: 0.000492


In [ ]:
print(f"Classic PageRank MAE: {classic_mae}")
print(f"Classic PageRank RMSE: {classic_rmse}")

print(f"Heat Kernel PageRank MAE: {heat_kernel_mae}")
print(f"Heat Kernel PageRank RMSE: {heat_kernel_rmse}")

Classic PageRank MAE: 9.324099274038117e-05
Classic PageRank RMSE: 0.0004917658863565371
Heat Kernel PageRank MAE: 9.324099274037658e-05
Heat Kernel PageRank RMSE: 0.0004917658863565372
